In [1]:
%load_ext autoreload
%autoreload 2
import utils.models as models  # noqa: F403
import utils.api as api

CACHED_HOURLY_DATA = "cached_hourly_data.json"
CACHED_FORCAST_DATA = "cached_forecast_data.json"
CACHED_RAW_DATA = "cached_raw_data.json"

FORCASTS = (CACHED_FORCAST_DATA, CACHED_HOURLY_DATA, CACHED_RAW_DATA)

In [2]:
# api.update_all_forecasts()

In [3]:
data = api.load_cached_data(CACHED_RAW_DATA)

In [4]:
m = models.GridpointGeoJson.model_validate(data)
m

GridpointGeoJson(geometry=GeoJsonGeometry(type='Polygon', coordinates=[[[-82.9828, 35.4538], [-82.9804, 35.4762], [-83.0078, 35.4782], [-83.0103, 35.4558], [-82.9828, 35.4538]]]), properties=Gridpoint(updateTime='2025-06-20T18:37:43+00:00', validTimes='2025-06-20T12:00:00+00:00/P8D', elevation=QuantitativeValue(value=868.9848, unitCode='wmoUnit:m'), forecastOffice='https://api.weather.gov/offices/GSP', gridId='GSP', gridX=40, gridY=68, temperature=GridpointQuantitativeValueLayer(uom='wmoUnit:degC', values=[{'validTime': '2025-06-20T12:00:00+00:00/PT1H', 'value': 18.333333333333332}, {'validTime': '2025-06-20T13:00:00+00:00/PT1H', 'value': 20}, {'validTime': '2025-06-20T14:00:00+00:00/PT1H', 'value': 21.666666666666668}, {'validTime': '2025-06-20T15:00:00+00:00/PT1H', 'value': 23.333333333333332}, {'validTime': '2025-06-20T16:00:00+00:00/PT1H', 'value': 24.444444444444443}, {'validTime': '2025-06-20T17:00:00+00:00/PT1H', 'value': 25.555555555555557}, {'validTime': '2025-06-20T18:00:00+0